#**Pre-request**

##Mount google drive


In [1]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Install pakages


In [2]:
#Install pakages
#%pip install -q -r /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/clustring_requirements.txt --no-cache-dir

project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%cd $project_path
%ls $project_path
!pip install -q -r "{project_path}requirement/Install/clustring_requirements.txt" --no-cache-dir



ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/requirement/install/clustring_requirements.txt'
/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
configs/  notebooks/  requirement/  run_experiment.py  src/
dataset/  README.md   results/      splits/            tests/


##Import  libs

In [3]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import logging
import yaml
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import altair as alt
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
import altair as alt
import numpy as np
import pandas as pd
import altair as alt

# Try the best transformer for large datasets
try:
    alt.data_transformers.enable("vegafusion")
    print("VegaFusion enabled for large datasets.")
except Exception:
    print("VegaFusion not available — falling back to disable max rows limit.")
    alt.data_transformers.disable_max_rows()



VegaFusion enabled for large datasets.


#Basic Methods

##Config

In [4]:

def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file and expand ${root_path} placeholders."""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)


    # --- Expand ${root_path} placeholders ---
    root = config.get("root_path", "")

    def expand_paths(obj):
        if isinstance(obj, dict):
            return {k: expand_paths(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [expand_paths(i) for i in obj]
        elif isinstance(obj, str) and "${root_path}" in obj:
            return obj.replace("${root_path}", root)
        else:
            return obj

    config = expand_paths(config)
    return config

## CDR dataset

In [5]:
def load_cdr(file_path, nrows=None):
    """Load a CSV file and safely parse datetime columns."""
    df = pd.read_csv(file_path, nrows=nrows)

    # Auto-detect and parse datetime columns
    for col in df.columns:
        if "datetime" in col.lower():
            df[col] = pd.to_datetime(df[col], errors="coerce")

    # Optional: strip extra spaces in headers
    df.columns = df.columns.str.strip()
    return df



def load_all_data(config):
    """
    Load all CSVs defined in config into a dict of DataFrames.
    Uses training.sample_size if available.
    """
    base = config["dataset"]["base_path"]
    files = config["dataset"]["files"]
    sample_size = config.get("training", {}).get("sample_size", None)

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path, nrows=sample_size)
        data[name] = df
    return data


#Clustring

##  t-SNE Helper Function

In [6]:
def run_tsne_per_dataset(data_raw):
    print("\n============================\n t-SNE per Dataset \n============================")
    for name, df in data_raw.items():
        print(f"\n📄 Running t-SNE for {name.upper()} dataset")

        # skip user merge if not needed
        if "label" not in df.columns:
            print(f"⚠️ Skipping {name}: no label column.")
            continue

        numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
        numeric_cols = [c for c in numeric_cols if c not in ["label"]]
        if len(numeric_cols) < 2:
            print(f"⚠️ Skipping {name}: only {len(numeric_cols)} numeric feature(s).")
            continue

        df = df.dropna(subset=["label"])
        df[numeric_cols] = df[numeric_cols].fillna(0)

        scaler = StandardScaler()
        scaled_feats = scaler.fit_transform(df[numeric_cols])
        tsne = TSNE(n_components=2, random_state=42, perplexity=30, learning_rate=200)
        emb = tsne.fit_transform(scaled_feats)
        df["tsne_0"], df["tsne_1"] = emb[:, 0], emb[:, 1]

        chart = (
            alt.Chart(df, title=f"{name.upper()} Dataset t-SNE")
            .mark_circle(size=60, opacity=0.7)
            .encode(
                x=alt.X("tsne_0", title="t-SNE Dimension 1"),
                y=alt.Y("tsne_1", title="t-SNE Dimension 2"),
                color=alt.Color("label:N", title="Fraud Label", scale=alt.Scale(scheme="dark2")),
                tooltip=["phone_no_m", "label"]
            )
            .properties(width=600, height=500)
            .interactive()
        )
        display(chart)


##Distribution analysis

In [7]:
def plot_fraud_distribution(
    data_raw_labeled,
    dataset_name,
    feature_col,
    title=None,
    top_n=None,
    min_users=3,
    size_field="fraud_users",
    color_field="fraud_ratio"
):
    """
    Generic fraud distribution analyzer for any categorical feature.

    Parameters
    ----------
    data_raw_labeled : dict[str, pd.DataFrame]
        Dictionary of labeled raw datasets (e.g., {"app": df_app, "voc": df_voc, ...})
    dataset_name : str
        Which dataset to use ("app", "voc", "sms", "user", etc.)
    feature_col : str
        The categorical column to group by (e.g., "busi_name", "calltype_id", "city_name")
    title : str, optional
        Custom chart title (default auto-generated)
    top_n : int or None
        Show only the top N by `size_field`. If None, show all.
    min_users : int
        Minimum unique users required to include a category (default 3)
    size_field : str
        Column to use for bubble size ("fraud_users", "users", or "fraud_ratio")
    color_field : str
        Column to use for bubble color ("fraud_ratio", "fraud_users", or "users")
    """

    import pandas as pd
    import altair as alt

    # --- Validation ---
    if dataset_name not in data_raw_labeled:
        print(f"❌ Dataset '{dataset_name}' not found.")
        return

    df = data_raw_labeled[dataset_name].copy()
    required_cols = {"phone_no_m", "label", feature_col}
    missing = required_cols - set(df.columns)
    if missing:
        print(f"❌ Missing columns in {dataset_name}: {missing}")
        return

    # --- Aggregate unique users and unique fraud users ---
    summary = (
        df.groupby(feature_col)
        .agg(
            users=("phone_no_m", "nunique"),
            fraud_users=("phone_no_m", lambda x: x[df.loc[x.index, "label"] == 1].nunique())
        )
        .reset_index()
    )

    summary["fraud_ratio"] = summary["fraud_users"] / summary["users"]
    summary = summary.query("users >= @min_users").fillna(0)

    # --- Sort and clip ---
    if size_field not in summary.columns:
        print(f"⚠️ '{size_field}' not found. Defaulting to 'fraud_users'.")
        size_field = "fraud_users"

    if color_field not in summary.columns:
        print(f"⚠️ '{color_field}' not found. Defaulting to 'fraud_ratio'.")
        color_field = "fraud_ratio"

    summary_sorted = summary.sort_values(size_field, ascending=False)
    summary_plot = summary_sorted.head(top_n) if top_n else summary_sorted

    # --- Title ---
    chart_title = title or f"Fraud Distribution by {feature_col} ({dataset_name.upper()})"

    # --- Visualization ---
    chart = (
        alt.Chart(summary_plot, title=chart_title)
        .mark_circle(opacity=0.8)
        .encode(
            x=alt.X("users:Q", title="Number of Users (log)", scale=alt.Scale(type="log")),
            y=alt.Y("fraud_ratio:Q", title="Fraud Ratio"),
            size=alt.Size(f"{size_field}:Q", title=size_field.replace("_", " ").title(),
                          scale=alt.Scale(range=[10, 800])),
            color=alt.Color(f"{color_field}:Q", title=color_field.replace("_", " ").title(),
                            scale=alt.Scale(scheme="reds")),
            tooltip=[feature_col, "users", "fraud_users", "fraud_ratio"]
        )
        .properties(width=650, height=500)
        .interactive()
    )

    display(chart)

    # --- Table Output ---
    print(f"\nTop {'All' if not top_n else top_n} '{feature_col}' by {size_field} in {dataset_name.upper()}:")
    display(summary_plot)

    return summary_plot


## ⚙️ Load Config and Data

##RAW

###T-SNE

In [8]:
# =====================================================
# 📂 Load RAW Datasets
# =====================================================
config = load_config("configs/baseline.yaml")

raw_cfg = config["clustring"]["raw"]
raw_base = raw_cfg["base_path"]
raw_files = {k: os.path.join(raw_base, v) for k, v in raw_cfg["files"].items()}

# --- Verify existence
for name, path in raw_files.items():
    if not os.path.exists(path):
        print(f"⚠️ Warning: RAW file not found for {name}: {path}")

# --- Load
data_raw = {k: pd.read_csv(p) for k, p in raw_files.items()}

print("\n✅ Loaded RAW datasets:")
for name, df in data_raw.items():
    print(f"RAW {name}: {df.shape}")



/tmp/ipython-input-1836586913.py:16: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_raw = {k: pd.read_csv(p) for k, p in raw_files.items()}



✅ Loaded RAW datasets:
RAW app: (3283602, 4)
RAW sms: (6848509, 4)
RAW user: (6106, 13)
RAW voc: (5015430, 8)


In [9]:

# =====================================================
# 🏷️ Merge labels once for all non-user datasets
# =====================================================


user_labels = data_raw["user"][["phone_no_m", "label"]]

data_raw_labeled = {}
for name, df in data_raw.items():
    if name == "user":
        data_raw_labeled[name] = df
        continue
    merged_df = df.merge(user_labels, on="phone_no_m", how="left")
    data_raw_labeled[name] = merged_df

for name, df in data_raw_labeled.items():
    if name == "user":
        continue
    total = len(df)
    labeled = df["label"].notna().sum()
    print(f"{name}: {labeled}/{total} rows labeled ({labeled/total:.2%})")

print("\n✅ Merged fraud labels into all datasets.")

app: 3283602/3283602 rows labeled (100.00%)
sms: 6848509/6848509 rows labeled (100.00%)
voc: 5015430/5015430 rows labeled (100.00%)

✅ Merged fraud labels into all datasets.


###Distribution

In [10]:


# =====================================================
# 🧠 Run standard t-SNE per dataset (fraud vs legit)
# =====================================================
print("\n============================\n t-SNE per Dataset \n============================")
#run_tsne_per_dataset(data_raw_labeled)

plot_fraud_distribution(data_raw_labeled, "app", "busi_name", size_field="users", color_field="fraud_ratio", title="Top 20 Apps by Fraud Volume and Risk")
plot_fraud_distribution(data_raw_labeled, "sms", "calltype_id", size_field="users", color_field="fraud_ratio", title="Top 20 SMS  type by Fraud Volume and Risk")
plot_fraud_distribution(data_raw_labeled, "voc", "calltype_id", size_field="users", color_field="fraud_ratio", title="Top 20 Voice call type by Fraud Volume and Risk")
plot_fraud_distribution(data_raw_labeled, "voc", "city_name", size_field="users", color_field="fraud_ratio", title="Top 20 voice city name by Fraud Volume and Risk")



 t-SNE per Dataset 


ImportError: The "vegafusion" data transformer and chart.transformed_data feature requires
version 1.5.0 or greater of the 'vegafusion-python-embed' and 'vegafusion' packages.
These can be installed with pip using:
    pip install "vegafusion[embed]>=1.5.0"
Or with conda using:
    conda install -c conda-forge "vegafusion-python-embed>=1.5.0" "vegafusion>=1.5.0"

ImportError: No module named 'vegafusion'

alt.Chart(...)


Top All 'busi_name' by users in APP:


,busi_name,users,fraud_users,fraud_ratio
709,DNS,4956,1375,0.277441
4454,网页浏览,4571,999,0.218552
3539,微信,4472,1059,0.236807
4537,腾讯网,4267,1062,0.248887
1674,QQ,4214,950,0.225439
...,...,...,...,...
1381,Mobiett,3,1,0.333333
3966,水浒Q传,3,1,0.333333
2109,TmallBox,3,0,0.000000
4899,面包旅行,3,2,0.666667


...

ImportError: The "vegafusion" data transformer and chart.transformed_data feature requires
version 1.5.0 or greater of the 'vegafusion-python-embed' and 'vegafusion' packages.
These can be installed with pip using:
    pip install "vegafusion[embed]>=1.5.0"
Or with conda using:
    conda install -c conda-forge "vegafusion-python-embed>=1.5.0" "vegafusion>=1.5.0"

ImportError: No module named 'vegafusion'

alt.Chart(...)


Top All 'city_name' by users in VOC:


,city_name,users,fraud_users,fraud_ratio
9,成都,3281,928,0.282841
8,德阳,759,149,0.196311
14,绵阳,732,125,0.170765
13,眉山,654,100,0.152905
16,资阳,640,99,0.154688
3,南充,515,89,0.172816
0,乐山,488,103,0.211066
18,遂宁,484,88,0.181818
1,内江,431,75,0.174014
20,雅安,425,46,0.108235


,city_name,users,fraud_users,fraud_ratio
9,成都,3281,928,0.282841
8,德阳,759,149,0.196311
14,绵阳,732,125,0.170765
13,眉山,654,100,0.152905
16,资阳,640,99,0.154688
3,南充,515,89,0.172816
0,乐山,488,103,0.211066
18,遂宁,484,88,0.181818
1,内江,431,75,0.174014
20,雅安,425,46,0.108235


#Freez

In [11]:
%pip freeze > "{project_path}requirement/freez/clustring_requirements-lock.txt"
